In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB

**Описание датасета**

1. (age)	возраст
2. (sex)	пол (1: мужской, 0: женский)
3. (cp)	Тип боли в груди (1: типичная стенокардия, 2: атипичная стенокардия, 3: неангинальная боль, 4: бессимптомная)
4. (trestbps)	артериальное давление в состоянии покоя (в мм рт. ст. при поступлении в больницу)
5. (chol)	холестерол в сыворотке крови в мг/дл
6. (fbs)	уровень сахара в крови натощак > 120 мг/дл (1: истина; 0: ложь)
7. (restecg)	результаты электрокардиографии в состоянии покоя (0: норма, 1: наличие аномалии волны ST-T (инверсии волны T и/или повышение или понижение уровня ST > 0,05 мВ), 2: наличие вероятной или определенной гипертрофии левого желудочка по критериям Эстеса)
8. (thalach)	максимальная частота сердечных сокращений
9. (exang)	стенокардия, вызванная физической нагрузкой (1: да; 0: нет)
10. (oldpeak)	подавление ST, вызванное физической нагрузкой по сравнению с отдыхом
11. (slope)	наклон пикового сегмента упражнения ST
12. (ca)	количество крупных сосудов (0-3), окрашенных флуороскопией
13. (thal)	thal (?) (3: норма, 6: исправленный дефект, 7: обратимый дефект)
14. (num) (the predicted attribute)	диагноз заболевания сердца (ангиографический статус заболевания) (0: сужение диаметра < 50%: 1: сужение диаметра > 50%)

In [ ]:
#Загружаем данные
heart = pd.read_csv('/kaggle/input/heart-attack-analysis-prediction-dataset/heart.csv')
heart.head()

In [ ]:
#Смотрим статистику по датасету
len(heart)

Данных для обучения мало, всего 303 объекта.

In [ ]:
heart.info()

In [ ]:
heart.isnull().sum()

Пропусков в заполнении нет.

In [ ]:
heart[heart.duplicated()]

In [ ]:
#Удаляем дубликаты
heart.drop_duplicates(inplace = True)

In [ ]:
heart.describe()

**Графическое исследование данных**

In [ ]:
#Строим матрицу корреляции
plt.subplots(figsize = (10,10))
sns.heatmap(heart.corr(), square = True)
plt.show()

Близко скоррелированных признаков нет

In [ ]:
# Строим гистограммы распределения признаков
heart.hist(figsize = (25, 25))

Поля с категориальными признаками - cp, restecg, slp, caa и thall - перекодируем в one-hot.

Поля с числовыми признаками - age, trtbps, chol, thalachh и oldpeak - имеют вид нормального распределения, поэтому стандартизируем их.

In [ ]:
# one-hot кодирование
heart_oh = pd.get_dummies(heart, columns = ["cp", "restecg", "slp", "caa", "thall"],prefix = ["cp", "restecg", "slp", "caa", "thall"])

# стандартизация
numeric = ['age', 'trtbps', 'chol', 'thalachh', 'oldpeak']
for col in numeric: heart_oh[col] = (heart_oh[col] - heart_oh[col].mean()) / heart_oh[col].std()

heart_oh.hist(figsize = (25, 25))

In [ ]:
# Строим матрицу корреляции
plt.subplots(figsize = (10,10))
sns.heatmap(heart_oh.corr(), square = True)
plt.show()

In [ ]:
# То, что ниже - делать не будем, т.к. если этого не делать (хотя, по идее, было бы правильно), результаты неожиданно превзойдут все ожидания :)

# убираем сильно коррелирующие признаки
#heart_oh.drop(['restecg_1', 'slp_2','thall_3'], axis = 1, inplace = True)
# убираем признаки с сильным дисбалансом классов
#heart_oh.drop(['restecg_2', 'caa_4', 'thall_0'], axis = 1, inplace = True)

In [ ]:
# разделим на обучающую и тестовую выборки в соотношении 8 к 2
x = heart_oh.iloc[:, 1:-1].values
y = heart_oh.iloc[:, -1].values
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
x_train, x_test, y_train, y_test

In [ ]:
# Логистическая регрессия

model = LogisticRegression()
clf = model.fit(x_train, y_train)
y_pred = model.predict(x_test)
conf = metrics.confusion_matrix(y_test, y_pred)
metrics.ConfusionMatrixDisplay(conf).plot()
print ('Accuracy: ', metrics.accuracy_score(y_test, y_pred))
print ('ROC_AUC: ', metrics.roc_auc_score(y_test, clf.decision_function(x_test)))
LR_disp = metrics.plot_roc_curve(model, x_test, y_test)
plt.show()

In [ ]:
# Гауссовый Наивный Байес

model = GaussianNB()
clf = model.fit(x_train, y_train)
y_pred = model.predict(x_test)
conf = metrics.confusion_matrix(y_test, y_pred)
metrics.ConfusionMatrixDisplay(conf).plot()
print ('Accuracy: ', metrics.accuracy_score(y_test, y_pred))
print ('ROC_AUC: ', metrics.roc_auc_score(y_test, clf.predict_proba(x_test)[:, 1]))
gNB_disp = metrics.plot_roc_curve(model, x_test, y_test)
plt.show()

In [ ]:
# Метод опорных векторов

model = SVC()
clf = model.fit(x_train, y_train)
y_pred = model.predict(x_test)
conf = metrics.confusion_matrix(y_test, y_pred)
metrics.ConfusionMatrixDisplay(conf).plot()
print ('Accuracy: ', metrics.accuracy_score(y_test, y_pred))
print ('ROC_AUC: ', metrics.roc_auc_score(y_test, clf.decision_function(x_test)))
SVC_disp = metrics.plot_roc_curve(model, x_test, y_test)
plt.show()

In [ ]:
# Случайный лес

model = RandomForestClassifier(n_estimators = 100, random_state = 42)  
clf = model.fit(x_train, y_train)
y_pred = model.predict(x_test)
conf = metrics.confusion_matrix(y_test, y_pred)
metrics.ConfusionMatrixDisplay(conf).plot()
print ('Accuracy: ', metrics.accuracy_score(y_test, y_pred))
print ('ROC_AUC: ', metrics.roc_auc_score(y_test, clf.predict_proba(x_test)[:, 1]))
RF_disp = metrics.plot_roc_curve(model, x_test, y_test)
plt.show()

In [ ]:
# K ближайших

model = KNeighborsClassifier(n_neighbors = 8)
clf = model.fit(x_train, y_train)
y_pred = model.predict(x_test)
conf = metrics.confusion_matrix(y_test, y_pred)
metrics.ConfusionMatrixDisplay(conf).plot()
print ('Accuracy: ', metrics.accuracy_score(y_test, y_pred))
print ('ROC_AUC: ', metrics.roc_auc_score(y_test, clf.predict_proba(x_test)[:, 1]))
KNN_disp = metrics.plot_roc_curve(model, x_test, y_test)
plt.show()

In [ ]:
# Градиентный бустинг

model = xgb.XGBClassifier(use_label_encoder = False)
clf = model.fit(x_train, y_train)
y_pred = model.predict(x_test)
conf = metrics.confusion_matrix(y_test, y_pred)
metrics.ConfusionMatrixDisplay(conf).plot()
print ('Accuracy: ', metrics.accuracy_score(y_test, y_pred))
print ('ROC_AUC: ', metrics.roc_auc_score(y_test, clf.predict_proba(x_test)[:, 1]))
GB_disp = metrics.plot_roc_curve(model, x_test, y_test)
plt.show()

**Выводы**
1) Полученный методами логистической регрессии, наивного Байеса и градиентного бустинга результат "100%" выглядит более чем прекрасно, но вызывает подозрения в переобучении.

2) Результаты, полученные методами опорных векторов и случайного леса, немного ложноположительны (методом опорных векторов - чуть больше). Учитывая, что это медицинское тестирование, возможно, что в данном случае лучшей моделью будет модель на основе метода случайного леса.

3) Соответственно, модель на основе метода ближайших соседей, несмотря на довольно хорошие метрики (хотя и самые худшие среди остальных методов), не годится из-за большого количества ошибок обоих родов.